 Детальная и комплексная кластеризация спиртных напитков объемом 0.7 л, которые продаются в сети ["КуулКлевер"](https://www.coolclever.ru/catalog/otdokhni/all?filters=11099%255B0%255D%3D10%2611099%255B1%255D%3D11%2611099%255B2%255D%3D9%2611099%255B3%255D%3D12%2611099%255B4%255D%3D17%2611099%255B5%255D%3D13%2611099%255B6%255D%3D14%2611099%255B7%255D%3D15%2611099%255B8%255D%3D16%2611099%255B9%255D%3D43%2611109%255B0%255D%3D0.700&sort=cheap&page=9), с помощью различных методов машинного обучения.


In [343]:
# Объявляю нужные библиотеки
import pandas as pd # Библиотека для работы с фреймворками
from sklearn.preprocessing import LabelEncoder # Объект разметки текстовых данных

In [344]:
# Загрузка датасета из csv-файла
df = pd.read_csv('CoolClever.csv', sep=',')
df

,Напиток,Цена,Страна,Крепость,Выдержка,Подарочная упаковка
0,Водка,490,Россия,40.0,0.0,0
1,Водка,490,Россия,40.0,0.0,0
2,Водка,490,Россия,40.0,0.0,0
3,Водка,539,Россия,40.0,0.0,0
4,Водка,539,Россия,40.0,0.0,0
...,...,...,...,...,...,...
189,Коньяк,12900,Франция,40.0,10.0,1
190,Арманьяк,19000,Франция,51.5,17.0,1
191,Арманьяк,20000,Франция,52.0,20.0,1
192,Коньяк,20000,Франция,40.0,10.0,1


In [345]:
# Разметка текстовых данных в таблице
le = LabelEncoder()
df['Напиток'] = le.fit_transform(df['Напиток'])
df['Страна'] = le.fit_transform(df['Страна'])
df

,Напиток,Цена,Страна,Крепость,Выдержка,Подарочная упаковка
0,4,490,7,40.0,0.0,0
1,4,490,7,40.0,0.0,0
2,4,490,7,40.0,0.0,0
3,4,539,7,40.0,0.0,0
4,4,539,7,40.0,0.0,0
...,...,...,...,...,...,...
189,7,12900,10,40.0,10.0,1
190,0,19000,10,51.5,17.0,1
191,0,20000,10,52.0,20.0,1
192,7,20000,10,40.0,10.0,1


In [346]:
# Информация о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Напиток              194 non-null    int64  
 1   Цена                 194 non-null    int64  
 2   Страна               194 non-null    int64  
 3   Крепость             194 non-null    float64
 4   Выдержка             194 non-null    float64
 5   Подарочная упаковка  194 non-null    int64  
dtypes: float64(2), int64(4)
memory usage: 9.2 KB


In [347]:
# Описание датасета
df.describe()

,Напиток,Цена,Страна,Крепость,Выдержка,Подарочная упаковка
count,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000
mean,4.546392,4818.237113,8.587629,41.374227,5.615979,0.474227
std,2.808280,4348.584582,3.678118,5.130116,6.470996,0.500627
min,0.000000,490.000000,0.000000,17.000000,0.000000,0.000000
25%,3.000000,1811.500000,7.000000,40.000000,1.000000,0.000000
50%,4.000000,3275.000000,10.000000,40.000000,3.000000,0.000000
75%,7.000000,6000.000000,10.000000,41.925000,10.000000,1.000000
max,11.000000,21500.000000,13.000000,58.900000,40.000000,1.000000


In [348]:
# Корреляционные зависимости
corr = df.corr()
corr

,Напиток,Цена,Страна,Крепость,Выдержка,Подарочная упаковка
Напиток,1.000000,-0.059029,-0.275535,-0.330971,-0.266388,-0.059955
Цена,-0.059029,1.000000,0.299034,0.394579,0.761954,0.646128
Страна,-0.275535,0.299034,1.000000,0.316342,0.365786,0.343117
Крепость,-0.330971,0.394579,0.316342,1.000000,0.251846,0.327776
Выдержка,-0.266388,0.761954,0.365786,0.251846,1.000000,0.542724
Подарочная упаковка,-0.059955,0.646128,0.343117,0.327776,0.542724,1.000000
